In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',100)

from sklearn.preprocessing import LabelEncoder;
from itertools import product

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from xgboost import XGBRegressor
from xgboost import plot_importance

def plot_features(booster,figsize):
    fig,ax = plt.subplot(1,1,figsize=figsize)
    return plot_importance(booster,ax =ax)

import time
import gc
import sys
import pickle
sys.version_info

In [ ]:
item = pd.read_csv("../input/competitive-data-science-predict-future-sales/items.csv")
train = pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
shops = pd.read_csv("../input/competitive-data-science-predict-future-sales/shops.csv")
cat = pd.read_csv("../input/competitive-data-science-predict-future-sales/item_categories.csv")

test = pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv").set_index("ID")

In [ ]:
train.head()

In [ ]:
plt.figure(figsize=(10,4))
plt.xlim(-100,3000)
sns.boxplot(x =train.item_cnt_day)

plt.figure(figsize=(10,4))
plt.xlim(train.item_price.min(),train.item_price.max()*1.1)
sns.boxplot(x = train.item_price)

In [ ]:
#Removing the outliers
train = train[train.item_price<100000]
train = train[train.item_cnt_day<1001]

In [ ]:
train.loc[train.item_price<0]

We found out that index 484683 has a item price below 0. We will faill these value with median

In [ ]:
median =train.loc[(train.shop_id==32)&(train.item_id==2973)&(train.date_block_num==4)&(train.item_price>0)].item_price.median()

In [ ]:
train.loc[train.item_price<0,'item_price'] = median


In [ ]:
shops.head()

As we can see, some of the shops are duplicate. So, we would fix its name for train and test set

In [ ]:
train.loc[train.shop_id==0,"shop_id"]=57
test.loc[test.shop_id==0,"shop_id"]=57

train.loc[train.shop_id==1,"shop_id"]=58
test.loc[test.shop_id==1,"shop_id"]=58

train.loc[train.shop_id==10,"shop_id"]=11
test.loc[test.shop_id==10,"shop_id"]=11

In [ ]:
cat.head(25)

In [ ]:
cat['split'] = cat['item_category_name'].str.split('-')
cat['type'] = cat['split'].map(lambda x: x[0].strip())
cat['type_code'] = LabelEncoder().fit_transform(cat['type'])

#If subtype is a nan then

cat['subtype'] = cat['split'].map(lambda x: x[1].strip() if len(x)>1 else x[0].strip())
cat["subtype_code"] = LabelEncoder().fit_transform(cat['subtype'])
cat = cat[["item_category_id","type_code","subtype_code"]]
item.drop(["item_name"],axis=1,inplace=True)


In [ ]:
shops.nunique()


Monthly  Sales

In [ ]:
len(list(set(test.item_id) - set(test.item_id).intersection(set(train.item_id)))),len(list(set(test.item_id))), len(test)

In [ ]:
ts = time.time()
matrix =[]
cols =["date_block_num","shop_id","item_id"]

for i in range(34):
    sales = train[train.date_block_num==i]
    matrix.append(np.array(list(product([i],sales.shop_id.unique(),sales.item_id.unique())),dtype="int16"))
    
matrix = pd.DataFrame(np.vstack(matrix),columns =cols)
#to reduce size
matrix["date_block_num"] = matrix['date_block_num'].astype('int8')
matrix["shop_id"] = matrix['shop_id'].astype(np.int8)
matrix["item_id"] = matrix["item_id"].astype(np.int16)
matrix.sort_values(cols,inplace=True)
time.time()-ts
    
    

In [ ]:
matrix.nunique()
train['revenue'] = train['item_price']* train['item_cnt_day']

In [ ]:
%%time
group = train.groupby(['date_block_num','shop_id','item_id']).agg({"item_cnt_day":["sum"]})
group.columns =['item_cnt_month']
group.reset_index=True

matrix = pd.merge(matrix,group,on=cols,how='left')
matrix['item_cnt_month'] = (matrix['item_cnt_month'].fillna(0).clip(0,20).astype(np.float16))

Test Set

In [ ]:
test['date_block_num'] = 34
test['date_block_num'] = test['date_block_num'].astype(np.int8)
test['shop_id']  = test['shop_id'].astype(np.int8)
test['item_id'] = test['item_id'].astype(np.int16)

In [ ]:
%%time
matrix = pd.concat([matrix,test],ignore_index=True,sort=False,keys=cols)
matrix.fillna(0,inplace=True)

In [ ]:
%%time
matrix = pd.merge(matrix,shops,on=['shop_id'],how='left')
matrix = pd.merge(matrix,item,on=['item_id'],how='left')
matrix = pd.merge(matrix,cat,on=['item_category_id'],how='left')

matrix['item_category_id'] = matrix['item_category_id'].astype(np.int8)
matrix['type_code'] = matrix['type_code'].astype(np.int8)
matrix['subtype_code'] = matrix['subtype_code'].astype(np.int8)

In [ ]:
matrix.head()

Target Lags

In [ ]:
def lag_features(df,lags,col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id',col +"_lag_"+ str(i)]
        shifted['date_block_num']+= i
        df = pd.merge(df,shifted,on=["date_block_num","shop_id","item_id"],how='left')
    return df

In [ ]:
matrix = lag_features(matrix,[1,2,3,6,12], "item_cnt_month")
matrix.head()

**Mean Encoded Features**

In [ ]:
%%time
group = matrix.groupby(['date_block_num']).agg({"item_cnt_month":['mean']})
group.columns = ['date_avg_item_cnt']
group.reset_index(inplace=True)

matrix = pd.merge(matrix,group,on=['date_block_num'],how='left')
matrix['date_avg_item_cnt'] = matrix['date_avg_item_cnt'].astype(np.int8)
matrix = lag_features(matrix,[1],"date_avg_item_cnt")

matrix.drop(['date_avg_item_cnt'],axis=1,inplace=True)


In [ ]:
%%time
group = matrix.groupby(['date_block_num','item_id']).agg({'item_cnt_month':['mean']})
group.columns = ['date_item_avg_count']
group.reset_index(inplace=True)

matrix = pd.merge(matrix,group,on=['date_block_num',"item_id"],how='left')
matrix['date_item_avg_count'] = matrix['date_item_avg_count'].astype(np.float16)
matrix = lag_features(matrix,[1,2,3,6,12],"date_item_avg_count")

matrix.drop(["date_item_avg_count"],axis=1,inplace=True)

In [ ]:
%%time
group = matrix.groupby(["date_block_num","shop_id"]).agg({"item_cnt_month":['mean']})
group.columns = ['date_shop_avg_count']
group.reset_index(inplace=True)

matrix = pd.merge(matrix,group,on=["date_block_num","shop_id"],how='left')
matrix["date_shop_avg_count"] = matrix["date_shop_avg_count"].astype(np.float16)
matrix = lag_features(matrix,[1,2,3,6,12],"date_shop_avg_count")

matrix.drop(["date_shop_avg_count"],axis=1,inplace=True)

In [ ]:
%%time
group = matrix.groupby(["date_block_num","shop_id","item_category_id"]).agg({"item_cnt_month":["mean"]})
group.columns = ["date_shop_cat_avg_count"]
group.reset_index(inplace=True)

matrix = pd.merge(matrix,group,on=["date_block_num","shop_id","item_category_id"],how="left")
matrix["date_shop_cat_avg_count"] = matrix["date_shop_cat_avg_count"].astype(np.float16)
matrix = lag_features(matrix,[1],"date_shop_cat_avg_count")
matrix.drop(["date_shop_cat_avg_count"],axis=1,inplace=True)

In [ ]:
%%time
group = matrix.groupby(["date_block_num","item_category_id"]).agg({"item_cnt_month":['mean']})
group.columns = ['date_cat_avg_count']
group.reset_index(inplace=True)

matrix = pd.merge(matrix,group,on=["date_block_num","item_category_id"],how='left')
matrix["date_cat_avg_count"] = matrix["date_cat_avg_count"].astype(np.float16)
matrix = lag_features(matrix,[1],"date_cat_avg_count")

matrix.drop(["date_cat_avg_count"],axis=1,inplace=True)

In [ ]:
%%time
group = matrix.groupby(["date_block_num","type_code"]).agg({"item_cnt_month":['mean']})
group.columns = ['date_type_avg_count']
group.reset_index(inplace=True)

matrix = pd.merge(matrix,group,on=["date_block_num","type_code"],how='left')
matrix["date_type_avg_count"] = matrix["date_type_avg_count"].astype(np.float16)
matrix = lag_features(matrix,[1],"date_type_avg_count")

matrix.drop(["date_type_avg_count"],axis=1,inplace=True)

In [ ]:
%%time
group = matrix.groupby(["date_block_num","subtype_code"]).agg({"item_cnt_month":['mean']})
group.columns = ['date_subtype_avg_count']
group.reset_index(inplace=True)

matrix = pd.merge(matrix,group,on=["date_block_num","subtype_code"],how='left')
matrix["date_subtype_avg_count"] = matrix["date_subtype_avg_count"].astype(np.float16)
matrix = lag_features(matrix,[1],"date_subtype_avg_count")

matrix.drop(["date_subtype_avg_count"],axis=1,inplace=True)

In [ ]:
%%time
group = matrix.groupby(["date_block_num","shop_id","type_code"]).agg({"item_cnt_month":['mean']})
group.columns = ['date_shop_type_avg_count']
group.reset_index(inplace=True)

matrix = pd.merge(matrix,group,on=["date_block_num","shop_id","type_code"],how='left')
matrix["date_shop_type_avg_count"] = matrix["date_shop_type_avg_count"].astype(np.float16)
matrix = lag_features(matrix,[1],"date_shop_type_avg_count")

matrix.drop(["date_shop_type_avg_count"],axis=1,inplace=True)

In [ ]:
%%time
group = matrix.groupby(["date_block_num","shop_id","subtype_code"]).agg({"item_cnt_month":['mean']})
group.columns = ['date_shop_subtype_avg_count']
group.reset_index(inplace=True)

matrix = pd.merge(matrix,group,on=["date_block_num","shop_id","subtype_code"],how='left')
matrix["date_shop_subtype_avg_count"] = matrix["date_shop_subtype_avg_count"].astype(np.float16)
matrix = lag_features(matrix,[1],"date_shop_subtype_avg_count")

matrix.drop(["date_shop_subtype_avg_count"],axis=1,inplace=True)

In [ ]:
matrix

In [ ]:
matrix["date_avg_item_cnt_lag_1"] = matrix["date_avg_item_cnt_lag_1"].astype(np.float16)

In [ ]:
 #missed date shop_id cattegor

In [ ]:
#matrix.drop(["date_shop_avg_count_lag_1_y","date_shop_avg_count_lag_2_y","date_shop_avg_count_lag_3_y","date_shop_avg_count_lag_6_y","date_shop_avg_count_lag_12_y",],axis=1,inplace=True)

Trend Features

In [ ]:
group = train.groupby(['item_id']).agg({'item_price': ['mean']})
group.columns = ['item_avg_item_price']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['item_id'], how='left')
matrix['item_avg_item_price'] = matrix['item_avg_item_price'].astype(np.float16)

group = train.groupby(['date_block_num','item_id']).agg({'item_price': ['mean']})
group.columns = ['date_item_avg_item_price']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','item_id'], how='left')
matrix['date_item_avg_item_price'] = matrix['date_item_avg_item_price'].astype(np.float16)




In [ ]:
matrix.drop(["shop_name"],inplace=True,axis=1)

In [ ]:
matrix.info()

In [ ]:

lags = [1,2,3,4,5,6]
matrix = lag_features(matrix, lags, 'date_item_avg_item_price')

for i in lags:
    matrix['delta_price_lag_'+str(i)] = \
        (matrix['date_item_avg_item_price_lag_'+str(i)] - matrix['item_avg_item_price']) / matrix['item_avg_item_price']

def select_trend(row):
    for i in lags:
        if row['delta_price_lag_'+str(i)]:
            return row['delta_price_lag_'+str(i)]
    return 0



In [ ]:
matrix.tail()

In [ ]:
matrix

In [ ]:
%%time
matrix['delta_price_lag'] = matrix.apply(select_trend, axis=1)
matrix['delta_price_lag'] = matrix['delta_price_lag'].astype(np.float16)
matrix['delta_price_lag'].fillna(0, inplace=True)

# https://stackoverflow.com/questions/31828240/first-non-null-value-per-row-from-a-list-of-pandas-columns/31828559
# matrix['price_trend'] = matrix[['delta_price_lag_1','delta_price_lag_2','delta_price_lag_3']].bfill(axis=1).iloc[:, 0]
# Invalid dtype for backfill_2d [float16]




In [ ]:
fetures_to_drop = ['item_avg_item_price', 'date_item_avg_item_price']
for i in lags:
    fetures_to_drop += ['date_item_avg_item_price_lag_'+str(i)]
    fetures_to_drop += ['delta_price_lag_'+str(i)]

matrix.drop(fetures_to_drop, axis=1, inplace=True)

In [ ]:
matrix.info()

Last month shop revenue

In [ ]:
%%time
group = train.groupby(["date_block_num","shop_id"]).agg({'revenue':["sum"]})
group.columns = ["date_shop_revenue"]
group.reset_index(inplace=True)

matrix = pd.merge(matrix,group,on=["date_block_num","shop_id"],how='left')
matrix["date_shop_revenue"] = matrix["date_shop_revenue"].astype(np.float32)




In [ ]:
group = group.groupby(["shop_id"]).agg({"date_shop_revenue":["mean"]})
group.columns = ["shop_avg_revenue"]
group.reset_index(inplace=True)

matrix = pd.merge(matrix,group,on=["shop_id"],how = "left")
matrix["shop_avg_revenue"] = matrix["shop_avg_revenue"].astype(np.float32)

matrix["delta_revenue"] = (matrix["date_shop_revenue"] - matrix["shop_avg_revenue"])/matrix["shop_avg_revenue"]

matrix["delta_revenue"] = matrix["delta_revenue"].astype(np.float32)

matrix = lag_features(matrix,[1],"delta_revenue")

matrix.drop(["date_shop_revenue","shop_avg_revenue","delta_revenue"],axis=1,inplace=True)

Special Features

In [ ]:
matrix["month"] = matrix["date_block_num"] % 12


In [ ]:
days = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])

#mapping, days to months
matrix["days"] = matrix["month"].map(days).astype(np.int8)

Months since the last sale for each shop/item pair and for item only

In [ ]:
ts = time.time()
cache = {}
matrix['item_shop_last_sale'] = -1
matrix['item_shop_last_sale'] = matrix['item_shop_last_sale'].astype(np.int8)
for idx, row in matrix.iterrows():    
    key = str(row.item_id)+' '+str(row.shop_id)
    if key not in cache:
        if row.item_cnt_month!=0:
            cache[key] = row.date_block_num
    else:
        last_date_block_num = cache[key]
        matrix.at[idx, 'item_shop_last_sale'] = row.date_block_num - last_date_block_num
        cache[key] = row.date_block_num         
time.time() - ts

In [ ]:
matrix.info()

In [ ]:
matrix

In [ ]:
ts = time.time()
cache = {}
matrix['item_last_sale'] = -1
matrix['item_last_sale'] = matrix['item_last_sale'].astype(np.int8)
for idx, row in matrix.iterrows():    
    key = row.item_id
    if key not in cache:
        if row.item_cnt_month!=0:
            cache[key] = row.date_block_num
    else:
        last_date_block_num = cache[key]
        if row.date_block_num>last_date_block_num:
            matrix.at[idx, 'item_last_sale'] = row.date_block_num - last_date_block_num
            cache[key] = row.date_block_num         
time.time() - ts

Months since the first sale for each shop/item pair and for item only.

In [ ]:
%%time
matrix["item_shop_first_sale"] = matrix["date_block_num"] - matrix.groupby(["shop_id","item_id"])["date_block_num"].transform("min")
matrix["item_first_sale"] = matrix["date_block_num"] - matrix.groupby("item_id")["date_block_num"].transform("min")


**FINal Preperations**

In [ ]:
matrix = matrix[matrix.date_block_num>11]


Producing lags brings a lot of nulls.

In [ ]:
matrix

In [ ]:
%%time
def fill_na(df):
    for col in df.columns:
        if("_lag_" in col) & (df[col].isnull().any()):
            if("item_cnt" in col):
                df[col].fillna(0,inplace=True)
                
    return df

matrix = fill_na(matrix)
    

In [ ]:
matrix.columns

In [ ]:
matrix.info()

In [ ]:
matrix.to_pickle("data.pk1")
del matrix
del cache
del cat
del shops
del train
 

In [ ]:
data = pd.read_pickle("./data.pk1")

select perfect features

In [ ]:
data = data[['date_block_num', 'shop_id', 'item_id', 'item_cnt_month',
       'item_category_id', 'type_code', 'subtype_code', 'item_cnt_month_lag_1',
       'item_cnt_month_lag_2', 'item_cnt_month_lag_3', 'item_cnt_month_lag_6',
       'item_cnt_month_lag_12', 'date_avg_item_cnt_lag_1',
       'date_item_avg_count_lag_1', 'date_item_avg_count_lag_2',
       'date_item_avg_count_lag_3', 'date_item_avg_count_lag_6',
       'date_item_avg_count_lag_12', 'date_shop_avg_count_lag_1',
       'date_shop_avg_count_lag_2', 'date_shop_avg_count_lag_3',
       'date_shop_avg_count_lag_6', 'date_shop_avg_count_lag_12',
       'date_shop_cat_avg_count_lag_1', 'date_cat_avg_count_lag_1',
       #'date_type_avg_count_lag_1', 'date_subtype_avg_count_lag_1',
       #'date_shop_type_avg_count_lag_1', 
        #'date_shop_subtype_avg_count_lag_1',
       'delta_price_lag', 'delta_revenue_lag_1', 'month', 'days',
       'item_shop_last_sale', 'item_last_sale', 'item_shop_first_sale',
       'item_first_sale']]

In [ ]:
data

In [ ]:
X_train = data[data.date_block_num <33].drop(["item_cnt_month"],axis=1)
Y_train = data[data.date_block_num <33]["item_cnt_month"]

X_valid = data[data.date_block_num==33].drop(["item_cnt_month"],axis=1)
Y_valid = data[data.date_block_num==33]["item_cnt_month"]

X_test = data[data.date_block_num>33].drop(["item_cnt_month"],axis=1)

In [ ]:
del data
gc.collect();

In [ ]:
%%time

model = XGBRegressor(
    max_depth = 8,n_estimators=1000,
    min_child_weight=300,colsample_bytree=0.8,
    subsample=0.8,eta=0.3,seed=42)

model.fit(X_train,Y_train,eval_metric="rmse",eval_set=[(X_train,Y_train),(X_valid, Y_valid)],
         verbose =True,early_stopping_rounds=10)



In [ ]:
%%time
Y_pred = model.predict(X_valid).clip(0,20)
Y_test = model.predict(X_test).clip(0,20)

submission = pd.DataFrame({"ID":test.index,"item_cnt_month":Y_test})

submission.to_csv("xgb_submission",index=False)

# save predictions for an ensemble
pickle.dump(Y_pred, open('xgb_train.pickle', 'wb'))
pickle.dump(Y_test, open('xgb_test.pickle', 'wb'))

In [ ]:
def plot_features(booster, figsize):    
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax)

In [ ]:
plot_features(model, (10,14))